<a href="https://colab.research.google.com/github/MeghaGhosh/seq2seq/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive

/content/drive


In [4]:
%cd 'My Drive'

/content/drive/My Drive


In [5]:
%cd 'Chat Bot'

/content/drive/My Drive/Chat Bot


In [6]:
!ls

intents.json


In [12]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import json
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [14]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)


45 documents
9 classes ['course_search', 'faculty_search', 'goodbye', 'greeting', 'options', 'student_details', 'student_details_search', 'thanks', 'time_table']
74 unique stemmed words ["'s", ',', 'a', 'al', 'anyon', 'ar', 'avail', 'awesom', 'be', 'by', 'bye', 'can', 'chat', 'check', 'could', 'cours', 'dat', 'day', 'detail', 'do', 'doe', 'entry', 'facul', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'how', 'i', 'id', 'inform', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nam', 'next', 'nic', 'of', 'off', 'op', 'provid', 'rel', 'result', 'search', 'see', 'show', 'stud', 'support', 'tabl', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'up', 'want', 'what', 'which', 'you']


In [0]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = training[:,0]
train_y = training[:,1]

In [0]:
input_texts = []
target_texts = []
input_characters = []
target_characters = []
input_texts.append(train_x)
target_texts.append(train_y)
for char in input_texts:
  input_characters.append(char)
for char in target_texts:
  target_characters.append(char)
    


In [0]:
latent_dim = 256

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in train_x])
max_decoder_seq_length = max([len(txt) for txt in train_y])

encoder_inputs = Input(shape=(45,1))
encoder_LSTM = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(45,1))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, input_shape = (1,),activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
train_x = np.asarray(train_x, dtype=object)
train_y=np.asarray(train_y,dtype=object)

In [0]:
train_x=train_x.reshape((1,45,1))
train_y=train_y.reshape((1,45,1))

In [207]:
model.fit([train_x, train_y], train_y,
          batch_size=64,
          epochs=100)

Epoch 1/100


ValueError: ignored